# 🎄 NOEL PERAKENDE SATIŞ ANALİZİ VE TESLİMAT TAHMİNİ PROJESİ

---

## 📋 Proje Hakkında

Bu Jupyter Notebook, **Christmas Retail Sales and Marketing Analysis Dataset** kullanılarak hazırlanmış akademik bir veri bilimi çalışmasıdır.

### 🎯 Projenin Amacı
E-ticaret lojistik süreçleri üzerinden, **teslimat gecikmesini (Late/OnTime)** tahmin eden bir makine öğrenmesi modeli geliştirmek.

### 📊 Veri Seti
- **Kaynak:** Kaggle - Christmas Retail Sales Dataset
- **Dönem:** Black Friday, Cyber Monday ve Noel dönemi
- **Tablolar:** OrderHeader, OrderLine, Product, Promotion, Fulfillment, Returns, Calendar

### 🔬 İzlenecek Adımlar
1. Veri Tanıma
2. Veri Ön İşleme
3. Özellik Mühendisliği
4. Keşifsel Veri Analizi (EDA)
5. Makine Öğrenmesi
6. Model Değerlendirme
7. Sonuç ve Yorum

---

## 1. 📚 KÜTÜPHANELERİN YÜKLENMESİ

Projede kullanacağımız tüm Python kütüphanelerini yüklüyoruz.

In [ ]:
# =============================================================================
# TEMEL VERİ İŞLEME KÜTÜPHANELERİ
# =============================================================================
# NumPy: Sayısal hesaplamalar için temel kütüphane
import numpy as np

# Pandas: Veri analizi ve manipülasyonu için ana kütüphane
import pandas as pd

# =============================================================================
# GÖRSELLEŞTİRME KÜTÜPHANELERİ
# =============================================================================
# Matplotlib: Python'un temel grafik kütüphanesi
import matplotlib.pyplot as plt

# Seaborn: İstatistiksel görselleştirme kütüphanesi
import seaborn as sns

# =============================================================================
# İSTATİSTİK KÜTÜPHANELERİ
# =============================================================================
from scipy import stats
from scipy.stats import shapiro, ttest_ind, f_oneway, pearsonr

# =============================================================================
# MAKİNE ÖĞRENMESİ KÜTÜPHANELERİ
# =============================================================================
# Veri bölme
from sklearn.model_selection import train_test_split, cross_val_score

# Veri ön işleme
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Değerlendirme metrikleri
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

# Sınıflandırma algoritmaları
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# =============================================================================
# AYARLAR
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

print('✅ Tüm kütüphaneler başarıyla yüklendi!')

---

## 2. 📥 VERİ TANIMI VE YÜKLENMESİ

### Tablolar ve İş Sürecindeki Rolleri:
| Tablo | Açıklama |
|-------|----------|
| **OrderHeader** | Sipariş başlık bilgileri (müşteri, kanal, ödeme) |
| **OrderLine** | Sipariş satır detayları (ürün, miktar, fiyat) |
| **Product** | Ürün kataloğu (kategori, maliyet, tedarikçi) |
| **Promotion** | Kampanya bilgileri |
| **Fulfillment** | Sevkiyat ve teslimat (🎯 Hedef değişken burada) |
| **Returns** | İade kayıtları |
| **Calendar** | Takvim ve tatil bilgileri |

In [ ]:
# =============================================================================
# VERİ YÜKLEME
# =============================================================================
dosya_yolu = 'Christmas_Retail_Sales_and_Marketing_Analysis_Dataset.xlsx'

print('📂 Veri dosyası yükleniyor...')
print('='*60)

# Tüm tabloları yüklüyoruz
order_header = pd.read_excel(dosya_yolu, sheet_name='OrderHeader')
print(f'✅ OrderHeader: {order_header.shape[0]:,} satır, {order_header.shape[1]} sütun')

order_line = pd.read_excel(dosya_yolu, sheet_name='OrderLine')
print(f'✅ OrderLine: {order_line.shape[0]:,} satır, {order_line.shape[1]} sütun')

product = pd.read_excel(dosya_yolu, sheet_name='Product')
print(f'✅ Product: {product.shape[0]:,} satır, {product.shape[1]} sütun')

promotion = pd.read_excel(dosya_yolu, sheet_name='Promotion')
print(f'✅ Promotion: {promotion.shape[0]:,} satır, {promotion.shape[1]} sütun')

# Ana tablo - Hedef değişken (DeliveryStatus) burada
fulfillment = pd.read_excel(dosya_yolu, sheet_name='Fulfillment')
print(f'✅ Fulfillment: {fulfillment.shape[0]:,} satır, {fulfillment.shape[1]} sütun')

returns = pd.read_excel(dosya_yolu, sheet_name='Returns')
print(f'✅ Returns: {returns.shape[0]:,} satır, {returns.shape[1]} sütun')

calendar = pd.read_excel(dosya_yolu, sheet_name='Calendar')
print(f'✅ Calendar: {calendar.shape[0]:,} satır, {calendar.shape[1]} sütun')

print('\n' + '='*60)
print('📊 TÜM TABLOLAR BAŞARIYLA YÜKLENDİ!')

In [ ]:
# =============================================================================
# FULFILLMENT TABLOSU İNCELEME (ANA TABLO)
# =============================================================================
# Bu tablo hedef değişkenimiz olan DeliveryStatus'u içerir

print('🚚 FULFILLMENT TABLOSU DETAYLARI')
print('='*60)
print()
print('Sütun Açıklamaları:')
print('• ShipmentID: Sevkiyat kimlik numarası')
print('• OrderID: Sipariş numarası')
print('• Carrier: Kargo firması (FedEx, UPS, USPS, DHL)')
print('• ServiceLevel: Hizmet seviyesi (Express, Std, NextDay)')
print('• PromisedDate: Taahhüt edilen teslimat tarihi')
print('• ShipDate: Gönderim tarihi')
print('• DeliveryDate: Gerçek teslimat tarihi')
print('• ShipCost$: Kargo maliyeti')
print('• DeliveryStatus: 🎯 HEDEF DEĞİŞKEN (Late/OnTime)')
print()
fulfillment.head()

---

## 3. 🔧 VERİ ÖN İŞLEME

### 3.1 Tarih Alanlarının Dönüştürülmesi
Tarih alanlarını datetime formatına çeviriyoruz.

In [ ]:
# =============================================================================
# TARİH ALANLARININ DÖNÜŞTÜRÜLMESI
# =============================================================================
print('📅 Tarih alanları dönüştürülüyor...')

# Fulfillment tablosundaki tarih alanları
fulfillment['PromisedDate'] = pd.to_datetime(fulfillment['PromisedDate'], errors='coerce')
fulfillment['ShipDate'] = pd.to_datetime(fulfillment['ShipDate'], errors='coerce')
fulfillment['DeliveryDate'] = pd.to_datetime(fulfillment['DeliveryDate'], errors='coerce')
print('✅ Fulfillment tarihleri dönüştürüldü')

# OrderHeader tablosundaki tarih
order_header['OrderDateTime'] = pd.to_datetime(order_header['OrderDateTime'], errors='coerce')
print('✅ OrderHeader tarihleri dönüştürüldü')

# Calendar tablosu
calendar['Date'] = pd.to_datetime(calendar['Date'], errors='coerce')
print('✅ Calendar tarihleri dönüştürüldü')

print('\n✅ TÜM TARİH ALANLARI DÖNÜŞTÜRÜLDÜ!')

### 3.2 Tabloların Birleştirilmesi (Merge)

Analiz için gerekli tabloları birleştiriyoruz.

In [ ]:
# =============================================================================
# TABLOLARIN BİRLEŞTİRİLMESİ
# =============================================================================
print('🔗 Tablolar birleştiriliyor...')

# Adım 1: Fulfillment + OrderHeader
df = fulfillment.merge(
    order_header[['OrderID', 'OrderDateTime', 'ChannelID', 'PaymentType']],
    on='OrderID',
    how='left'
)
print(f'✅ Fulfillment + OrderHeader: {df.shape}')

# Adım 2: OrderLine bilgisi (her sipariş için ilk satır)
order_line_first = order_line.groupby('OrderID').first().reset_index()
df = df.merge(
    order_line_first[['OrderID', 'ProductID', 'Qty', 'UnitPrice', 'LineRevenue$']],
    on='OrderID',
    how='left'
)
print(f'✅ + OrderLine: {df.shape}')

# Adım 3: Product bilgisi
df = df.merge(
    product[['ProductID', 'Category', 'Subcategory', 'Vendor']],
    on='ProductID',
    how='left'
)
print(f'✅ + Product: {df.shape}')

print(f'\n📊 BİRLEŞTİRİLMİŞ VERİ SETİ: {df.shape[0]:,} satır, {df.shape[1]} sütun')
df.head()

---

## 4. 🛠️ ÖZELLİK MÜHENDİSLİĞİ (Feature Engineering)

Mevcut değişkenlerden yeni ve anlamlı değişkenler türetiyoruz:
- **Taşıma Süresi** = DeliveryDate - ShipDate
- **Taahhüt Süresi** = PromisedDate - ShipDate
- **Gecikme Günü** = DeliveryDate - PromisedDate
- Tarih özellikleri (ay, hafta, gün)

In [ ]:
# =============================================================================
# YENİ ÖZELLİKLERİN TÜRETİLMESİ
# =============================================================================
print('🛠️ Yeni özellikler türetiliyor...')

# 1. TAŞIMA SÜRESİ (Teslimat - Gönderim)
# Kargo firmasının performansını gösterir
df['Tasima_Suresi'] = (df['DeliveryDate'] - df['ShipDate']).dt.days
print(f'✅ Taşıma Süresi - Ortalama: {df["Tasima_Suresi"].mean():.2f} gün')

# 2. TAAHHÜT SÜRESİ (Taahhüt - Gönderim)
# Müşteriye söylenen süre
df['Taahhut_Suresi'] = (df['PromisedDate'] - df['ShipDate']).dt.days
print(f'✅ Taahhüt Süresi - Ortalama: {df["Taahhut_Suresi"].mean():.2f} gün')

# 3. GECİKME GÜN SAYISI (Teslimat - Taahhüt)
# Pozitif = Gecikmeli, Negatif = Erken
df['Gecikme_Gun'] = (df['DeliveryDate'] - df['PromisedDate']).dt.days
print(f'✅ Gecikme Günü - Ortalama: {df["Gecikme_Gun"].mean():.2f} gün')

# 4. TARİH ÖZELLİKLERİ
df['Gonderim_Ay'] = df['ShipDate'].dt.month
df['Gonderim_Hafta'] = df['ShipDate'].dt.isocalendar().week.astype(int)
df['Gonderim_Gun'] = df['ShipDate'].dt.dayofweek  # 0=Pazartesi
df['Hafta_Sonu'] = (df['Gonderim_Gun'] >= 5).astype(int)
print('✅ Tarih özellikleri eklendi')

# 5. TATİL DÖNEMİ BAYRAKLARI
df['Black_Friday_Donemi'] = ((df['Gonderim_Ay'] == 11) & (df['ShipDate'].dt.day >= 22)).astype(int)
df['Noel_Oncesi'] = ((df['Gonderim_Ay'] == 12) & (df['ShipDate'].dt.day <= 24)).astype(int)
print('✅ Tatil dönemi bayrakları eklendi')

print('\n✅ TÜM ÖZELLİKLER TÜRETİLDİ!')

---

## 5. 📊 KEŞİFSEL VERİ ANALİZİ (EDA)

### 5.1 Teslimat Durumu Dağılımı
Hedef değişkenimiz olan DeliveryStatus'un dağılımını inceliyoruz.

In [ ]:
# =============================================================================
# HEDEF DEĞİŞKEN DAĞILIMI
# =============================================================================
print('📊 TESLİMAT DURUMU DAĞILIMI')
print('='*60)

# Late / OnTime dağılımı
teslimat_dagilim = df['DeliveryStatus'].value_counts()
teslimat_oran = df['DeliveryStatus'].value_counts(normalize=True) * 100

print('Sayısal Dağılım:')
print(teslimat_dagilim)
print()
print('Yüzdesel Dağılım:')
for durum, oran in teslimat_oran.items():
    print(f'   {durum}: %{oran:.2f}')

# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
colors = ['#28a745', '#dc3545']  # OnTime=yeşil, Late=kırmızı
axes[0].bar(teslimat_dagilim.index, teslimat_dagilim.values, color=colors)
axes[0].set_title('Teslimat Durumu Dağılımı', fontsize=14)
axes[0].set_xlabel('Teslimat Durumu')
axes[0].set_ylabel('Sipariş Sayısı')
for i, v in enumerate(teslimat_dagilim.values):
    axes[0].text(i, v + 500, f'{v:,}', ha='center', fontsize=12)

# Pie chart
axes[1].pie(teslimat_dagilim.values, labels=teslimat_dagilim.index, autopct='%1.1f%%',
            colors=colors, explode=[0.02, 0.02], shadow=True)
axes[1].set_title('Teslimat Durumu Oranları', fontsize=14)

plt.tight_layout()
plt.show()

### 5.2 Kargo Firmalarına Göre Gecikme Oranları

In [ ]:
# =============================================================================
# KARGO FİRMALARINA GÖRE GECİKME ORANLARI
# =============================================================================
print('🚚 KARGO FİRMALARINA GÖRE GECİKME ANALİZİ')
print('='*60)

# Her kargo firması için gecikme oranı
kargo_gecikme = df.groupby('Carrier')['DeliveryStatus'].apply(
    lambda x: (x == 'Late').sum() / len(x) * 100
).sort_values(ascending=False)

print('Gecikme Oranları:')
for kargo, oran in kargo_gecikme.items():
    print(f'   {kargo}: %{oran:.2f}')

# Görselleştirme
plt.figure(figsize=(10, 6))
bars = plt.bar(kargo_gecikme.index, kargo_gecikme.values, color='coral')
plt.axhline(y=kargo_gecikme.mean(), color='red', linestyle='--', label=f'Ortalama: %{kargo_gecikme.mean():.1f}')
plt.title('Kargo Firmalarına Göre Gecikme Oranları', fontsize=14)
plt.xlabel('Kargo Firması')
plt.ylabel('Gecikme Oranı (%)')
plt.legend()
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'%{bar.get_height():.1f}', ha='center')
plt.tight_layout()
plt.show()

### 5.3 Hizmet Seviyelerine Göre Teslimat Performansı

In [ ]:
# =============================================================================
# HİZMET SEVİYELERİNE GÖRE PERFORMANS
# =============================================================================
print('📦 HİZMET SEVİYELERİNE GÖRE TESLİMAT PERFORMANSI')
print('='*60)

# ServiceLevel bazında analiz
servis_analiz = df.groupby('ServiceLevel').agg({
    'DeliveryStatus': lambda x: (x == 'Late').sum() / len(x) * 100,
    'Tasima_Suresi': 'mean',
    'ShipCost$': 'mean'
}).round(2)
servis_analiz.columns = ['Gecikme Oranı (%)', 'Ort. Taşıma Süresi (Gün)', 'Ort. Kargo Maliyeti ($)']

print(servis_analiz)

# Görselleştirme
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].bar(servis_analiz.index, servis_analiz['Gecikme Oranı (%)'], color='salmon')
axes[0].set_title('Gecikme Oranı')
axes[0].set_ylabel('%')

axes[1].bar(servis_analiz.index, servis_analiz['Ort. Taşıma Süresi (Gün)'], color='steelblue')
axes[1].set_title('Ortalama Taşıma Süresi')
axes[1].set_ylabel('Gün')

axes[2].bar(servis_analiz.index, servis_analiz['Ort. Kargo Maliyeti ($)'], color='green')
axes[2].set_title('Ortalama Kargo Maliyeti')
axes[2].set_ylabel('$')

plt.tight_layout()
plt.show()

### 5.4 Zaman ve Sezon Etkisi

In [ ]:
# =============================================================================
# ZAMAN VE SEZON ETKİSİ
# =============================================================================
print('📅 ZAMAN VE SEZON ETKİSİ ANALİZİ')
print('='*60)

# Haftalık gecikme trendi
haftalik = df.groupby('Gonderim_Hafta')['DeliveryStatus'].apply(
    lambda x: (x == 'Late').sum() / len(x) * 100
)

plt.figure(figsize=(14, 5))
plt.plot(haftalik.index, haftalik.values, marker='o', linewidth=2, color='coral')
plt.axhline(y=haftalik.mean(), color='red', linestyle='--', alpha=0.7)
plt.fill_between(haftalik.index, haftalik.values, alpha=0.3, color='coral')
plt.title('Haftalık Gecikme Oranı Trendi', fontsize=14)
plt.xlabel('Hafta Numarası')
plt.ylabel('Gecikme Oranı (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Tatil dönemlerinde gecikme
print('\n🎄 Tatil Dönemlerinde Gecikme Oranları:')
print(f"   Black Friday Dönemi: %{df[df['Black_Friday_Donemi']==1]['DeliveryStatus'].apply(lambda x: x=='Late').mean()*100:.2f}")
print(f"   Noel Öncesi: %{df[df['Noel_Oncesi']==1]['DeliveryStatus'].apply(lambda x: x=='Late').mean()*100:.2f}")
print(f"   Normal Dönem: %{df[(df['Black_Friday_Donemi']==0) & (df['Noel_Oncesi']==0)]['DeliveryStatus'].apply(lambda x: x=='Late').mean()*100:.2f}")

---

## 6. 🤖 MAKİNE ÖĞRENMESİ

### 6.1 Hedef Değişken Hazırlama

- **Problem Türü:** İkili Sınıflandırma (Binary Classification)
- **Hedef Değişken:** DeliveryStatus (Late=1, OnTime=0)

In [ ]:
# =============================================================================
# HEDEF DEĞİŞKEN HAZIRLIĞI
# =============================================================================
print('🎯 HEDEF DEĞİŞKEN HAZIRLANIYOR')
print('='*60)

# Late = 1 (Gecikmeli), OnTime = 0 (Zamanında)
df['Hedef'] = (df['DeliveryStatus'] == 'Late').astype(int)

print('Hedef Değişken Dağılımı:')
print(df['Hedef'].value_counts())
print(f"\nGecikme Oranı: %{df['Hedef'].mean()*100:.2f}")

### 6.2 Özellik Seçimi ve Veri Hazırlama

In [ ]:
# =============================================================================
# ÖZELLİK SEÇİMİ VE LABEL ENCODING
# =============================================================================
print('🔧 ÖZELLİKLER HAZIRLANIYOR')
print('='*60)

# Model için kullanılacak sütunlar
kategorik_sutunlar = ['Carrier', 'ServiceLevel', 'ChannelID', 'PaymentType', 'Category']
sayisal_sutunlar = ['Qty', 'UnitPrice', 'ShipCost$', 'Tasima_Suresi', 'Taahhut_Suresi',
                   'Gonderim_Ay', 'Gonderim_Gun', 'Hafta_Sonu']

# Eksik verileri temizle
model_df = df[kategorik_sutunlar + sayisal_sutunlar + ['Hedef']].dropna()
print(f'Temizlenmiş veri seti: {model_df.shape}')

# Label Encoding - Kategorik değişkenleri sayısala çevirme
le = LabelEncoder()
for sutun in kategorik_sutunlar:
    model_df[sutun + '_encoded'] = le.fit_transform(model_df[sutun].astype(str))
    print(f'✅ {sutun} encode edildi')

# Özellik matrisi oluşturma
encoded_sutunlar = [s + '_encoded' for s in kategorik_sutunlar]
ozellik_sutunlari = sayisal_sutunlar + encoded_sutunlar

X = model_df[ozellik_sutunlari]
y = model_df['Hedef']

print(f'\n📊 Özellik Matrisi (X): {X.shape}')
print(f'📊 Hedef Vektör (y): {y.shape}')

### 6.3 Eğitim ve Test Setlerine Ayırma

In [ ]:
# =============================================================================
# EĞİTİM VE TEST SETLERİNE AYIRMA
# =============================================================================
print('✂️ VERİ BÖLÜNÜYOR')
print('='*60)

# %80 eğitim, %20 test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print(f'Eğitim seti: {X_train.shape[0]:,} kayıt (%80)')
print(f'Test seti: {X_test.shape[0]:,} kayıt (%20)')

# Ölçeklendirme (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print('\n✅ Veriler ölçeklendirildi (StandardScaler)')

### 6.4 Lojistik Regresyon Modeli

In [ ]:
# =============================================================================
# LOJİSTİK REGRESYON MODELİ
# =============================================================================
print('📈 LOJİSTİK REGRESYON MODELİ')
print('='*60)

# Model oluşturma ve eğitme
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Tahmin
y_pred_log = log_reg.predict(X_test_scaled)
y_proba_log = log_reg.predict_proba(X_test_scaled)[:, 1]

# Sonuçlar
print(f'Eğitim Doğruluğu: %{log_reg.score(X_train_scaled, y_train)*100:.2f}')
print(f'Test Doğruluğu: %{accuracy_score(y_test, y_pred_log)*100:.2f}')
print(f'ROC-AUC Skoru: {roc_auc_score(y_test, y_proba_log):.4f}')

### 6.5 Karar Ağacı Modeli

In [ ]:
# =============================================================================
# KARAR AĞACI MODELİ
# =============================================================================
print('🌳 KARAR AĞACI MODELİ')
print('='*60)

# Model oluşturma ve eğitme
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_model.fit(X_train_scaled, y_train)

# Tahmin
y_pred_dt = dt_model.predict(X_test_scaled)
y_proba_dt = dt_model.predict_proba(X_test_scaled)[:, 1]

# Sonuçlar
print(f'Eğitim Doğruluğu: %{dt_model.score(X_train_scaled, y_train)*100:.2f}')
print(f'Test Doğruluğu: %{accuracy_score(y_test, y_pred_dt)*100:.2f}')
print(f'ROC-AUC Skoru: {roc_auc_score(y_test, y_proba_dt):.4f}')

---

## 7. 📊 MODEL DEĞERLENDİRME

### 7.1 Karışıklık Matrisi (Confusion Matrix)

In [ ]:
# =============================================================================
# KARIŞIKLIK MATRİSİ
# =============================================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lojistik Regresyon
cm_log = confusion_matrix(y_test, y_pred_log)
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['OnTime', 'Late'], yticklabels=['OnTime', 'Late'])
axes[0].set_title('Lojistik Regresyon - Karışıklık Matrisi', fontsize=12)
axes[0].set_xlabel('Tahmin')
axes[0].set_ylabel('Gerçek')

# Karar Ağacı
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['OnTime', 'Late'], yticklabels=['OnTime', 'Late'])
axes[1].set_title('Karar Ağacı - Karışıklık Matrisi', fontsize=12)
axes[1].set_xlabel('Tahmin')
axes[1].set_ylabel('Gerçek')

plt.tight_layout()
plt.show()

### 7.2 Sınıflandırma Raporu

In [ ]:
# =============================================================================
# SINIFLANDIRMA RAPORU
# =============================================================================
print('📋 LOJİSTİK REGRESYON - SINIFLANDIRMA RAPORU')
print('='*60)
print(classification_report(y_test, y_pred_log, target_names=['OnTime', 'Late']))

print('\n📋 KARAR AĞACI - SINIFLANDIRMA RAPORU')
print('='*60)
print(classification_report(y_test, y_pred_dt, target_names=['OnTime', 'Late']))

### 7.3 ROC Eğrisi

In [ ]:
# =============================================================================
# ROC EĞRİSİ KARŞILAŞTIRMASI
# =============================================================================
plt.figure(figsize=(10, 8))

# Lojistik Regresyon ROC
fpr_log, tpr_log, _ = roc_curve(y_test, y_proba_log)
auc_log = roc_auc_score(y_test, y_proba_log)
plt.plot(fpr_log, tpr_log, label=f'Lojistik Regresyon (AUC = {auc_log:.3f})', linewidth=2)

# Karar Ağacı ROC
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_proba_dt)
auc_dt = roc_auc_score(y_test, y_proba_dt)
plt.plot(fpr_dt, tpr_dt, label=f'Karar Ağacı (AUC = {auc_dt:.3f})', linewidth=2)

# Referans çizgisi
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Tahmin')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Eğrisi Karşılaştırması', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 7.4 Model Karşılaştırması

In [ ]:
# =============================================================================
# MODEL KARŞILAŞTIRMASI
# =============================================================================
print('📊 MODEL KARŞILAŞTIRMASI')
print('='*60)

sonuclar = pd.DataFrame({
    'Model': ['Lojistik Regresyon', 'Karar Ağacı'],
    'Doğruluk (%)': [accuracy_score(y_test, y_pred_log)*100, accuracy_score(y_test, y_pred_dt)*100],
    'ROC-AUC': [roc_auc_score(y_test, y_proba_log), roc_auc_score(y_test, y_proba_dt)]
}).round(3)

print(sonuclar.to_string(index=False))

# Görselleştirme
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(sonuclar))
width = 0.35

bars1 = ax.bar(x - width/2, sonuclar['Doğruluk (%)'], width, label='Doğruluk (%)', color='steelblue')
bars2 = ax.bar(x + width/2, sonuclar['ROC-AUC']*100, width, label='ROC-AUC (%)', color='coral')

ax.set_ylabel('Skor')
ax.set_title('Model Performans Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(sonuclar['Model'])
ax.legend()
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()

---

## 8. 📝 SONUÇ VE YORUM

### 8.1 Proje Özeti

In [ ]:
# =============================================================================
# PROJE SONUÇ RAPORU
# =============================================================================
print('='*60)
print('📊 PROJE SONUÇ RAPORU')
print('='*60)
print()
print('1️⃣ VERİ SETİ')
print(f'   • Toplam kayıt: {len(df):,}')
print(f'   • Kullanılan özellik sayısı: {len(ozellik_sutunlari)}')
print(f'   • Eğitim seti: {len(X_train):,} kayıt')
print(f'   • Test seti: {len(X_test):,} kayıt')
print()
print('2️⃣ MODEL PERFORMANSLARI')
print(f'   • Lojistik Regresyon: %{accuracy_score(y_test, y_pred_log)*100:.2f} doğruluk')
print(f'   • Karar Ağacı: %{accuracy_score(y_test, y_pred_dt)*100:.2f} doğruluk')
print()
print('3️⃣ ANA BULGULAR')
print('   • Teslimat gecikmesi kargo firması ve hizmet seviyesine göre değişmektedir')
print('   • Tatil dönemlerinde (Black Friday, Noel) gecikme oranları artmaktadır')
print('   • Express hizmet seviyesi en düşük gecikme oranına sahiptir')
print()
print('4️⃣ ÖNERİLER')
print('   • Yoğun dönemlerde ek kargo kapasitesi sağlanmalı')
print('   • Yüksek gecikmeli kargo firmalarıyla performans görüşmesi yapılmalı')
print('   • Kritik siparişler için Express hizmet önerilmeli')
print()
print('='*60)
print('✅ PROJE TAMAMLANDI!')
print('='*60)

---

## 📚 Kaynaklar

- Veri Seti: Kaggle - Christmas Retail Sales Dataset
- Scikit-learn Dokümantasyonu: https://scikit-learn.org/
- Pandas Dokümantasyonu: https://pandas.pydata.org/
- Seaborn Dokümantasyonu: https://seaborn.pydata.org/

---

**Hazırlayan:** Veri Bilimi Projesi
**Tarih:** 2025